In [1]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [5]:
config_file = "configs/resnet152_waymo.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2020-04-15 10:17:11,144 SSD INFO: Loaded configuration file configs/resnet152_waymo.yaml
2020-04-15 10:17:11,144 SSD INFO: Loaded configuration file configs/resnet152_waymo.yaml
2020-04-15 10:17:11,146 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet152'
        PRETRAINED: True
        OUT_CHANNELS:  [512, 1024, 512, 256, 256, 256]
        INPUT_CHANNELS: 3
    PRIORS:
    #TODO: change last one to [1,1]
        FEATURE_MAPS: [[34, 45], [17, 23], [9, 12], [5, 6], [3, 3], [2, 2]]
INPUT:
    IMAGE_WIDTH: 360 
    IMAGE_HEIGHT: 270
DATASETS:
    TRAIN: ("waymo_train",)
    TEST: ("waymo_val", )
DATA_LOADER:
  NUM_WORKERS: 8
SOLVER:
    MAX_ITER: 120000
    LR_STEPS: [80000, 100000]
    GAMMA: 0.1
    BATCH_SIZE: 16
    LR: 5e-3
EVAL_STEP: 5000
MODEL_SAVE_STEP: 2500
#should be resnet152_360x270
OUTPUT_DIR: 'outputs/resnet50_320x240'
DATASET_DIR: "datasets"
2020-04-15 10:17:11,146 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet152'
        P

In [ ]:
model = start_train(cfg)

resnet152
Detector initialized. Total Number of params:  72.44M
Backbone number of parameters: 71.24M
SSD Head number of parameters: 1.20M
2020-04-15 10:17:15,852 SSD.trainer INFO: No checkpoint found.
2020-04-15 10:17:15,852 SSD.trainer INFO: No checkpoint found.
17936
Dataset loaded. Subset: train, number of images: 14348
2020-04-15 10:17:17,578 SSD.trainer INFO: Start training ...
2020-04-15 10:17:17,578 SSD.trainer INFO: Start training ...
2020-04-15 10:17:27,985 SSD.trainer INFO: iter: 000010, lr: 0.00173, total_loss: 13.650 (13.650), reg_loss: 6.253 (6.253), cls_loss: 7.397 (7.397), time: 1.027 (1.027), eta: 1 day, 10:13:24, mem: 10290M
2020-04-15 10:17:27,985 SSD.trainer INFO: iter: 000010, lr: 0.00173, total_loss: 13.650 (13.650), reg_loss: 6.253 (6.253), cls_loss: 7.397 (7.397), time: 1.027 (1.027), eta: 1 day, 10:13:24, mem: 10290M
2020-04-15 10:17:35,137 SSD.trainer INFO: iter: 000020, lr: 0.00180, total_loss: 11.584 (12.617), reg_loss: 5.911 (6.082), cls_loss: 5.672 (6.535)

In [ ]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)